In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
noise_type = 'gaussian'      # 'gaussian', 'saltpepper', 'speckle'
model_type = 'conv'          # 'fc', 'conv'
epochs = 10
batch = 128
latent_dim = 64

In [ ]:
(train_x, _), (test_x, _) = tf.keras.datasets.mnist.load_data()
train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.
train_x = np.reshape(train_x, (-1, 28, 28, 1))
test_x = np.reshape(test_x, (-1, 28, 28, 1))


In [ ]:
def add_gaussian_noise(imgs, amount=0.5):
    noise = amount * np.random.normal(loc=0.0, scale=1.0, size=imgs.shape)
    return np.clip(imgs + noise, 0., 1.)

In [ ]:
def add_saltpepper_noise(imgs, amount=0.2):
    out = np.copy(imgs)
    num_salt = np.ceil(amount * imgs.size * 0.5)
    coords = [np.random.randint(0, i-1, int(num_salt)) for i in imgs.shape]
    out[tuple(coords)] = 1
    num_pepper = np.ceil(amount* imgs.size * 0.5)
    coords = [np.random.randint(0, i-1, int(num_pepper)) for i in imgs.shape]
    out[tuple(coords)] = 0
    return out

In [ ]:

def add_speckle_noise(imgs, amount=0.5):
    noise = amount * np.random.randn(*imgs.shape)
    return np.clip(imgs + imgs * noise, 0., 1.)

In [ ]:
if noise_type == 'gaussian':
    noisy_train = add_gaussian_noise(train_x)
    noisy_test = add_gaussian_noise(test_x)
elif noise_type == 'saltpepper':
    noisy_train = add_saltpepper_noise(train_x)
    noisy_test = add_saltpepper_noise(test_x)
elif noise_type == 'speckle':
    noisy_train = add_speckle_noise(train_x)
    noisy_test = add_speckle_noise(test_x)
else:
    noisy_train = train_x
    noisy_test = test_x

In [ ]:
if model_type == 'fc':
    flat_shape = np.prod(train_x.shape[1:])
    # Flatten input
    x_in = tf.keras.layers.Input(shape=(28, 28, 1))
    x = tf.keras.layers.Flatten()(x_in)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    encoded = tf.keras.layers.Dense(latent_dim, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(encoded)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(flat_shape, activation='sigmoid')(x)
    x_out = tf.keras.layers.Reshape((28,28,1))(x)
    autoencoder = tf.keras.Model(x_in, x_out)
    encoder = tf.keras.Model(x_in, encoded)
else:
    x_in = tf.keras.layers.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(x_in)
    x = tf.keras.layers.MaxPooling2D((2,2), padding='same')(x)
    x = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2,2), padding='same')(x)
    encoded = tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2,2))(encoded)
    x = tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(x)
    x_out = tf.keras.layers.Conv2D(1, (3,3), activation='sigmoid', padding='same')(x)
    autoencoder = tf.keras.Model(x_in, x_out)
    encoder = tf.keras.Model(x_in, encoded)

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
history = autoencoder.fit(
    noisy_train, train_x,
    epochs=epochs,
    batch_size=batch,
    shuffle=True,
    validation_data=(noisy_test, test_x)
)

In [ ]:
decoded_imgs = autoencoder.predict(noisy_test[:10])

plt.figure(figsize=(20, 8))
for i in range(10):
    a = plt.subplot(4, 10, i + 1)
    plt.imshow(test_x[i].reshape(28,28), cmap='gray')
    plt.title("Clean")
    plt.axis('off')
    
    b = plt.subplot(4, 10, i + 11)
    plt.imshow(noisy_test[i].reshape(28,28), cmap='gray')
    plt.title("Noisy")
    plt.axis('off')
    
    c = plt.subplot(4, 10, i + 21)
    plt.imshow(decoded_imgs[i].reshape(28,28), cmap='gray')
    plt.title("Denoised")
    plt.axis('off')
    
    # Visualize latent space (reshape if possible)
    d = plt.subplot(4, 10, i + 31)
    lat = encoder.predict(test_x[i:i+1])
    lat_img = lat.reshape(-1, lat.shape[-1]) if model_type == 'fc' else lat.reshape(-1, lat.shape[-1])
    plt.plot(lat_img[0])
    plt.title('Latent')
    plt.axis('off')
plt.tight_layout()
plt.show()